# ABS Quarterly Wage Price Index 6345

Note: the WPI follows price changes in a fixed "basket" of jobs.

## Python set-up

In [1]:
# system imports
import re
import textwrap

# analytic imports
import pandas as pd
from readabs import metacol

# local imports
from abs_helper import get_abs_data
from mgplot import (
    multi_start,
    series_growth_plot_finalise,
    line_plot_finalise,
)

# pandas display settings
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("6345.0")
YEARS, QUARTERS_PER_YEAR = 5, 4
plot_times = 0, -(QUARTERS_PER_YEAR * YEARS )- 1

In [3]:
textwrap.wrap(", ".join(abs_dict.keys()))

['634501, 634502a, 634502b, 634503a, 634503b, 634504a, 634504b, 634505a,',
 '634505b, 634507a, 634507b, 634508a, 634508b, 634509a, 634509b,',
 '63450Table2bto9b, 63450Table2ato9a']

## Plot

### Headline WPI for Australia

In [4]:
def headline() -> tuple[dict[str, pd.Series], str, str]:
    """Plot the headlines.
    Return a dictionary of the data plotted."""

    table = "634501"
    data = abs_dict[table]
    series_type = "Original"
    select = "Percentage Change from Corresponding Quarter of Previous Year"
    selected = meta[
        (meta[metacol.table] == table)
        & (meta[metacol.stype] == series_type)
        & meta[metacol.did].str.contains(select)
    ]

    text = "Total hourly rates of pay excluding bonuses ;  Australia"
    box = {}  # used for public v private comparative charts below

    for series_id, desc in zip(selected[metacol.id], selected[metacol.did]):
        # get the title
        title = (
            desc.replace(select, "")
            .replace(text, "")
            .replace(";", "")
            .replace("Private and Public", "All sectors")
            .strip()
        )
        title = re.sub(" +", " ", title)
        series = data[series_id].dropna()
        box[title] = series

        multi_start(
            series,
            function=line_plot_finalise,
            starts=plot_times,
            title=f"Annual Wage Price Growth: {title}",
            ylabel="Per cent per annum",
            rfooter=f"{source} {table}",
            lfooter=f'{text.replace(" ; ", ".")}. {series_type} series.',
            width=2,
            show=SHOW,
        )
    return box, text, series_type


headlines, name, stype = headline()

### WPI Public vs Private comparative charts

Around 77% of paid wages are in the private sector.

In [5]:
def pub_v_priv(box: dict[str, pd.Series], text: str, series_type: str) -> None:
    """Public v Private sector wage growth."""

    plot_data = pd.DataFrame(box)
    title = "Annual Wage Price Growth"
    multi_start(
        plot_data[plot_data.columns[:2]],
        function=line_plot_finalise,
        starts=plot_times,
        title=title,
        ylabel="Per cent per annum",
        rfooter=f"{source}",
        lfooter=f'{text.replace(" ; ", ".")}. {series_type} series.',
        width=2,
        show=SHOW,
    )


pub_v_priv(headlines, name, stype)

## WPI Growth Charts

In [6]:
def wpi_growth() -> None:
    """Plot WPI Growth."""

    table = "634501"
    data = abs_dict[table]

    for series_type in ("Original", "Seasonally Adjusted"):
        select = "Index"
        selected = meta[
            (meta[metacol.table] == table)
            & (meta[metacol.stype] == series_type)
            & meta[metacol.did].str.contains(select)
        ]
        for title, series_id in zip(selected[metacol.did], selected[metacol.id]):
            title = title.replace(
                "Quarterly Index ;  Total hourly rates "
                "of pay excluding bonuses ;  Australia ;  ",
                "",
            )
            title = title.replace(" ;", "")
            title = title.replace("  ", " ")
            type_txt = {"Original": "Orig", "Seasonally Adjusted": "Seas Adj"}[
                series_type
            ]
            title = f"WPI Growth: {title} ({type_txt})"
            series = data[series_id]
            series_growth_plot_finalise(
                series,
                plot_from=plot_times[1],
                tag="recent",
                title=title,
                rfooter=f"{source} {table}",
                lfooter=f"Australia. WPI = Wage Price Index. {series_type}. ",
                show=SHOW,
            )


wpi_growth()

## CPI vs WPI Comparison Charts

In [7]:
import readabs as ra
from readabs import metacol as mc
from mgplot import bar_plot_finalise

def cpi_wpi_comparison() -> None:
    """Compare CPI and WPI indices since Dec 2019."""
    
    # Get WPI index
    wpi_table = "634501"
    _, wpi_id, _ = ra.find_abs_id(meta, {
        wpi_table: mc.table,
        "Quarterly Index": mc.did,
        "Private and Public": mc.did,
        "All industries": mc.did,
        "Seasonally Adjusted": mc.stype,
    })
    wpi_index = abs_dict[wpi_table][wpi_id]
    
    # Fetch just the CPI table needed (single_excel_only avoids re-setting chart dir)
    cpi_dict, cpi_meta = ra.read_abs_cat("6401.0", single_excel_only="64010Appendix1a")
    cpi_table = "64010Appendix1a"
    _, cpi_id, _ = ra.find_abs_id(cpi_meta, {
        cpi_table: mc.table,
        "Index Numbers": mc.did,
        "All groups CPI, seasonally adjusted": mc.did,
    })
    cpi_index = cpi_dict[cpi_table][cpi_id]
    
    # Align data and drop rows where either is NaN
    base_period = pd.Period("2019Q4", freq="Q")
    combined = pd.DataFrame({"CPI": cpi_index, "WPI": wpi_index}).dropna()
    since_covid = combined[combined.index >= base_period]
    
    # Chart 1: Rebased index (Dec 2019 = 100)
    base_values = since_covid.loc[base_period]
    rebased = since_covid / base_values * 100
    line_plot_finalise(
        rebased,
        title="CPI and WPI since December 2019",
        ylabel=f"Index ({base_period} = 100)",
        width=2,
        rfooter=f"{source} 6401.0",
        lfooter="Australia. Seasonally Adjusted.",
        legend={"loc": "upper left", "fontsize": "small"},
        show=SHOW,
    )
    
    # Chart 2: QoQ growth (side by side bars)
    qoq = since_covid.pct_change(1, fill_method=None) * 100
    qoq = qoq.iloc[1:]  # drop first row (NaN from pct_change)
    bar_plot_finalise(
        qoq,
        title="CPI and WPI: Quarter-on-Quarter Growth",
        ylabel="Per cent",
        rfooter=f"{source} 6401.0",
        lfooter="Australia. Seasonally Adjusted.",
        legend={"loc": "best", "fontsize": "small"},
        y0=True,
        show=SHOW,
    )
    
    # Chart 3: TTY (through the year) growth - lines
    tty = since_covid.pct_change(4, fill_method=None) * 100
    tty = tty.iloc[4:]  # drop first 4 rows (NaN from pct_change)
    line_plot_finalise(
        tty,
        title="CPI and WPI: Through-the-Year Growth",
        ylabel="Per cent per annum",
        width=2,
        rfooter=f"{source} 6401.0",
        lfooter="Australia. Seasonally Adjusted.",
        legend={"loc": "best", "fontsize": "small"},
        y0=True,
        show=SHOW,
    )

    # Chart 4: Real WPI (WPI deflated by CPI) - full history
    real_wpi = combined["WPI"] / combined["CPI"] * 100
    real_wpi.name = "Real WPI"
    real_footer = "Australia. Seasonally Adjusted. Real WPI = WPI / CPI \u00d7 100."
    multi_start(
        real_wpi,
        function=line_plot_finalise,
        starts=plot_times,
        title="Real Wage Price Index",
        ylabel="Index (WPI/CPI \u00d7 100)",
        width=2,
        rfooter=f"{source} 6401.0",
        lfooter=real_footer,
        show=SHOW,
    )

cpi_wpi_comparison()

## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2026-02-19 07:06:38

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.9.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

mgplot : 0.2.18
pandas : 2.3.3
re     : 2.2.1
readabs: 0.1.8

Watermark: 2.6.0



In [9]:
print("Finished")

Finished
